In [3]:
import sys
import os
import numpy as np
import time
import json
import matplotlib.pyplot as plt
import pyfftw.interfaces as pyfftw
from datetime import datetime

pyfftw.cache.enable()

# --- Initial Conditions ---
def initial_rand_2D(X, Y):
    return 0.1 * np.random.standard_normal(size=X.shape)

def inital_amb_seperated(X, Y):
    base = -0.9 * np.ones_like(X)
    half = X.shape[0] // 2 + 16
    base[32:32 + half, :] = 1.0
    return 0.7 * base + 0.8 * initial_rand_2D(X, Y)

def initial_c_0_2D(X, Y, c_0=0.5, init_noise=0.001):
    return (2. * c_0 - 1.) + init_noise * np.random.standard_normal(size=X.shape)

def initial_dot_inner_outer_2D(X, Y, r, L, c_0_outside, c_0_inside):
    radius_mask = X**2 + Y**2 < (r * L)**2
    base = initial_c_0_2D(X, Y, c_0=c_0_outside)
    base[radius_mask] = (2. * c_0_inside - 1.)
    return base

# --- Main Solver ---
def solve_ambplus_2D(phi_0=None, c_0=0.4, t_state=0.0, t_len=100.0, tau=0.01,
                     eps_val=1., a=-0.25, b=0.25, lam_val=1.75, zeta=2.0, D=0.05, M=1.,
                     s_start=-32.*np.pi, s_end=32.*np.pi, s_N=200):

    # Create timestamped results directory
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    save_dir = os.path.join("npy_files_fig4", f"run_{timestamp}")
    os.makedirs(save_dir, exist_ok=True)

    log_file = os.path.join(save_dir, "log.csv")
    prev_iter = 0

    # Space setup
    L = s_end - s_start
    x = np.linspace(s_start, s_end, s_N, endpoint=False)
    y = np.linspace(s_start, s_end, s_N, endpoint=False)
    X, Y = np.meshgrid(x, y, indexing="ij")
    kx = np.fft.fftfreq(s_N, d=L/s_N) * 2 * np.pi
    ky = np.fft.fftfreq(s_N, d=L/s_N) * 2 * np.pi
    KX, KY = np.meshgrid(kx, ky, indexing="ij")
    K_2 = KX**2 + KY**2
    K_4 = K_2**2

    K_cutoff = 0.67 * K_2.max()
    dealiasing_mask = K_2 < K_cutoff**2
    dx = abs(X[1][0] - X[0][0])
    dy = abs(Y[0][1] - Y[0][0])
    dS = dx * dy

    # Initial field
    if phi_0 is None:
        phi = inital_amb_seperated(X, Y)
        initial_name = "inital_amb_seperated"
    else:
        phi = phi_0
        initial_name = "loaded_npy"
        if os.path.exists(log_file):
            log_data = np.loadtxt(log_file, delimiter=",", skiprows=1)
            prev_iter = int(log_data[-1, 0])
            t_state = log_data[-1, 1]

    # Save parameters
    with open(os.path.join(save_dir, "parameters.csv"), 'w') as f:
        f.write("c_0,t_state,t_len,tau,eps_val,a,b,lam_val,zeta,D,M,s_start,s_end,s_N\n")
        f.write(f"{c_0},{t_state},{t_len},{tau},{eps_val},{a},{b},{lam_val},{zeta},{D},{M},{s_start},{s_end},{s_N}\n")

    # Save metadata
    meta = {
        "c_0": c_0,
        "t_state": t_state,
        "t_len": t_len,
        "tau": tau,
        "eps_val": eps_val,
        "a": a,
        "b": b,
        "lam_val": lam_val,
        "zeta": zeta,
        "D": D,
        "M": M,
        "s_start": s_start,
        "s_end": s_end,
        "s_N": s_N,
        "initial_condition": initial_name
    }
    with open(os.path.join(save_dir, "meta.json"), "w") as f:
        json.dump(meta, f, indent=4)

    f_phi = pyfftw.numpy_fft.fft2(phi, norm="backward", threads=8)

    t_N = round(t_len / tau)
    gaussian_scale = np.sqrt(tau / dS)
    check = 20000  # Save interval

    if not os.path.exists(log_file):
        with open(log_file, 'w') as f:
            f.write("iteration,time,total_mass,phi_min,phi_max\n")

    start = time.time()

    for ii in range(prev_iter, prev_iter + t_N):
        if ii % check == 0:
            end = time.time()
            print(f"Iteration {ii} took {end - start:.2f} s")

            phi_real = phi.real
            np.save(os.path.join(save_dir, f"phi_{ii:06d}.npy"), phi_real)
            print(f"Saved field to phi_{ii:06d}.npy")

            with open(log_file, 'a') as f:
                f.write(f"{ii},{t_state},{np.sum(phi_real)*dS / L**2},{np.min(phi_real)},{np.max(phi_real)}\n")

            start = time.time()

        phi_3 = phi**3
        dphi_dx = pyfftw.numpy_fft.ifft2(1j * KX * f_phi, norm="backward", threads=8)
        dphi_dy = pyfftw.numpy_fft.ifft2(1j * KY * f_phi, norm="backward", threads=8)
        laplacian_phi = pyfftw.numpy_fft.ifft2(-K_2 * f_phi, norm="backward", threads=8)
        lapl_phi_prod_grad_phi_x_fft = pyfftw.numpy_fft.fft2(laplacian_phi * dphi_dx, norm="backward", threads=8)
        lapl_phi_prod_grad_phi_y_fft = pyfftw.numpy_fft.fft2(laplacian_phi * dphi_dy, norm="backward", threads=8)

        grad_phi_2 = dphi_dx**2 + dphi_dy**2

        non_linear_term = -K_2 * pyfftw.numpy_fft.fft2(b * phi_3 + lam_val * grad_phi_2, norm="backward", threads=8)
        non_linear_term -= 1j * zeta * (KX * lapl_phi_prod_grad_phi_x_fft + KY * lapl_phi_prod_grad_phi_y_fft)
        non_linear_term *= dealiasing_mask

        white_noise_x_fft = pyfftw.numpy_fft.fft2(np.random.standard_normal(size=KX.shape), norm="backward", threads=8)
        white_noise_y_fft = pyfftw.numpy_fft.fft2(np.random.standard_normal(size=KY.shape), norm="backward", threads=8)
        gaussian_term = -np.sqrt(2 * D * M) * 1j * (KX * white_noise_x_fft + KY * white_noise_y_fft)

        f_phi = (f_phi + tau * M * non_linear_term + gaussian_scale * gaussian_term) / (1. + tau * M * (a * K_2 + eps_val * K_4))

        phi = pyfftw.numpy_fft.ifft2(f_phi, norm="backward", threads=8)
        t_state += tau

    # Final save
    phi_real = phi.real
    np.save(os.path.join(save_dir, f"phi_{ii+1:06d}.npy"), phi_real)
    plt.imsave(os.path.join(save_dir, "phi_final.png"), phi_real, cmap="inferno", vmin=-1.5, vmax=1.5)
    print(f"Saved final state to phi_{ii+1:06d}.npy and phi_final.png")

    with open(log_file, 'a') as f:
        f.write(f"{ii+1},{t_state},{np.sum(phi_real)*dS / L**2},{np.min(phi_real)},{np.max(phi_real)}\n")


# --- Main Entrypoint ---
def main():
    import argparse

    # Ignore Jupyter's --f argument
    args = [arg for arg in sys.argv[1:] if not arg.startswith('--f=')]

    if len(args) > 0:
        filename = args[0]
        try:
            phi_0 = np.load(filename)
        except Exception as e:
            print(f"Error reading file: {e}")
            sys.exit(1)
        print(f"Loaded {filename} with shape {phi_0.shape}")
        N = phi_0.shape[0]
    else:
        phi_0 = None
        N = 128

    np.random.seed(0)
    solve_ambplus_2D(phi_0, c_0=0.6, s_N=N, tau=0.02, t_len=10000, D=0.0, zeta=2.25, lam_val=1.8, s_start=-64, s_end=64)

if __name__ == "__main__":
    main()


Iteration 0 took 0.00 s
Saved field to phi_000000.npy
Iteration 20000 took 53.70 s
Saved field to phi_020000.npy
Iteration 40000 took 56.21 s
Saved field to phi_040000.npy
Iteration 60000 took 57.83 s
Saved field to phi_060000.npy
Iteration 80000 took 54.90 s
Saved field to phi_080000.npy
Iteration 100000 took 55.67 s
Saved field to phi_100000.npy
Iteration 120000 took 57.64 s
Saved field to phi_120000.npy
Iteration 140000 took 57.05 s
Saved field to phi_140000.npy
Iteration 160000 took 57.08 s
Saved field to phi_160000.npy
Iteration 180000 took 57.38 s
Saved field to phi_180000.npy
Iteration 200000 took 56.99 s
Saved field to phi_200000.npy
Iteration 220000 took 59.64 s
Saved field to phi_220000.npy
Iteration 240000 took 67.61 s
Saved field to phi_240000.npy
Iteration 260000 took 61.80 s
Saved field to phi_260000.npy
Iteration 280000 took 56.30 s
Saved field to phi_280000.npy
Iteration 300000 took 55.83 s
Saved field to phi_300000.npy
Iteration 320000 took 56.03 s
Saved field to phi_3